I have consistently been confused by how I can learn the dynamics of the time series, using my own time data points (in Anode and torchdiffeq repos).

In [2]:
from cosmoNODE.loaders import Anode as A

In [49]:
from cosmoNODE.anode.models import ODEBlock, ODEFunc, ODENet
from cosmoNODE.anode.training import Trainer

In [129]:
import numpy as np
from torchdiffeq import odeint_adjoint as odeint

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import DataLoader

In [4]:
a = A()
dataloader = DataLoader(a)

In [76]:
item = a[0]
lc = item[0]
obj_class = item[1]
lc

tensor([[[ 5.9750e+04, -5.4481e+02],
         [ 5.9750e+04, -8.1643e+02],
         [ 5.9750e+04, -4.7139e+02],
         [ 5.9750e+04, -3.8898e+02],
         [ 5.9752e+04, -6.8186e+02],
         [ 5.9752e+04, -1.0615e+03],
         [ 5.9752e+04, -5.2495e+02],
         [ 5.9752e+04, -3.9348e+02],
         [ 5.9752e+04, -3.5589e+02],
         [ 5.9767e+04, -5.4801e+02],
         [ 5.9767e+04, -8.1519e+02],
         [ 5.9767e+04, -4.7552e+02],
         [ 5.9767e+04, -4.0566e+02],
         [ 5.9767e+04, -4.2120e+02],
         [ 5.9770e+04, -5.5490e+02],
         [ 5.9770e+04, -8.2004e+02],
         [ 5.9770e+04, -4.7700e+02],
         [ 5.9770e+04, -4.0027e+02],
         [ 5.9770e+04, -4.1529e+02],
         [ 5.9779e+04, -6.3052e+02],
         [ 5.9779e+04, -9.2100e+02],
         [ 5.9779e+04, -5.1853e+02],
         [ 5.9779e+04, -4.2218e+02],
         [ 5.9779e+04, -4.2282e+02],
         [ 5.9782e+04, -2.8004e+02],
         [ 5.9782e+04, -4.4910e+02],
         [ 5.9782e+04, -3.1670e+02],
 

In [111]:
lc = lc.squeeze()
mjds = lc[:, 0]
fluxes = lc[:, 1]
zero = torch.cat((fluxes, mjds))
b = fluxes[0].reshape(1)
b.shape

torch.Size([1])

In [112]:
device = torch.device('cuda:' if torch.cuda.is_available() else 'cpu')
data_dim = 1 # the data dimension should be 1 (only giving model flux)
hidden_dim = 16
output_dim = 1

func = ODEFunc(device, data_dim=data_dim, hidden_dim=hidden_dim, augment_dim=0)


In [113]:

model = ODENet(device, data_dim=data_dim, hidden_dim=hidden_dim, output_dim=output_dim, time_dependent=False)


In [114]:
block = ODEBlock(device, odefunc=func)
print(block)


ODEBlock(
  (odefunc): ODEFunc(
    (fc1): Linear(in_features=1, out_features=16, bias=True)
    (fc2): Linear(in_features=16, out_features=16, bias=True)
    (fc3): Linear(in_features=16, out_features=1, bias=True)
    (non_linearity): ReLU(inplace)
  )
)


In [116]:
fwd = block(b, eval_times=mjds)
fwd.shape
fwd

tensor([[-5.4481e+02],
        [-5.4459e+02],
        [-5.4436e+02],
        [-5.4414e+02],
        [-4.9081e+02],
        [-4.9061e+02],
        [-4.9041e+02],
        [-4.9010e+02],
        [-4.8980e+02],
        [-2.2339e+02],
        [-2.2330e+02],
        [-2.2321e+02],
        [-2.2307e+02],
        [-2.2293e+02],
        [-1.9132e+02],
        [-1.9124e+02],
        [-1.9116e+02],
        [-1.9104e+02],
        [-1.9096e+02],
        [-1.1733e+02],
        [-1.1728e+02],
        [-1.1723e+02],
        [-1.1718e+02],
        [-1.1711e+02],
        [-1.0047e+02],
        [-1.0042e+02],
        [-1.0038e+02],
        [-1.0034e+02],
        [-1.0028e+02],
        [-4.3524e+01],
        [-4.3504e+01],
        [-4.3485e+01],
        [-4.3456e+01],
        [-4.3427e+01],
        [-3.6589e+01],
        [-3.6573e+01],
        [-3.6556e+01],
        [-3.6532e+01],
        [-3.6507e+01],
        [-2.4433e+01],
        [-2.4421e+01],
        [-2.4410e+01],
        [-2.4392e+01],
        [-2

In [92]:
# pred = model(fluxes)

RuntimeError: size mismatch, m1: [1 x 352], m2: [1 x 16] at /pytorch/aten/src/TH/generic/THTensorMath.cpp:961

In [130]:
odefunc = ODEFunc(device, data_dim, hidden_dim, augment_dim,
                  time_dependent, non_linearity)

self.odeblock = ODEBlock(device, odefunc, tol=tol, adjoint=adjoint)
self.linear_layer = nn.Linear(self.odeblock.odefunc.input_dim,
                              self.output_dim)


NameError: name 'augment_dim' is not defined

In [67]:
pred


tensor([-182.8846], grad_fn=<AddBackward0>)

In [119]:
true_y0 = torch.tensor([[2., 0.]])
true_y0.shape

torch.Size([1, 2])

In [123]:
true_y0 = torch.tensor([[2., 0.]])
t = torch.linspace(0., 25., 352)
true_A = torch.tensor([[-0.1, 2.0], [-2.0, -0.1]])


class Lambda(nn.Module):

    def forward(self, t, y):
        return torch.mm(y**3, true_A)


with torch.no_grad():
    true_y = odeint(Lambda(), true_y0, t, method='dopri5')

In [155]:
batch_time = 352  # max flux seq len
data_size = 1000
batch_size = 16


def get_batch():
    s = torch.from_numpy(np.random.choice(np.arange(50, dtype=np.int64), 1, replace=False))
    batch_y0 = true_y[s]  # (M, D)
    batch_t = t[:batch_time]  # (T)
    batch_y = torch.stack([true_y[s + i] for i in range(batch_time)], dim=0)  # (T, M, D)
    return batch_y0, batch_t, batch_y

In [188]:
np.random.choice(np.arange(50, dtype=np.int64), 2, replace=False)

array([ 8, 35])

In [156]:
batch_y0, batch_t, batch_y = get_batch()

IndexError: index 352 is out of bounds for dimension 0 with size 352